##### Author: Bishesh Kafle
##### Date : 2024-07-22

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# Define Kafka topic name and bootstrap server address
kafka_topic_name = 'Topic1'
kafka_bootstrap_servers = 'localhost:9092'

# Initialize a Spark session with Kafka support
spark = SparkSession \
    .builder \
    .appName("Structured Streaming") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .getOrCreate()

# Set logging level to ERROR to reduce verbosity
spark.sparkContext.setLogLevel("ERROR")

# Read streaming data from Kafka
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

# Print the schema of the incoming Kafka messages
df.printSchema()

# Select the 'value' and 'timestamp' fields from the Kafka message and cast 'value' to string
df1 = df.selectExpr("CAST(value AS STRING)", "timestamp")

# Define the schema of the incoming CSV data
df_schema_string = "order_id INT, account_number STRING, branch STRING, transaction_code STRING"

# Parse the 'value' field from CSV format into individual columns based on the schema
df2 = df1 \
    .select(from_csv(col("value"), df_schema_string) \
    .alias("data"), "timestamp")

# Flatten the 'data' structure to select individual fields along with the timestamp
df3 = df2.select("data.*", "timestamp")

# Create a temporary view to allow for SQL queries on the processed data
df3.createOrReplaceTempView("proc_rw_transaction_data")

# Execute an SQL query to select all data from the temporary view
data = spark.sql("SELECT * FROM proc_rw_transaction_data")

# Write the streaming query results to an in-memory table for further processing or visualization
data_agg_write_stream = data \
    .writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode("append") \
    .option("truncate", "false") \
    .format("memory") \
    .queryName("temp_stream_data") \
    .start()

# Await termination for a short period to allow the stream to start processing
data_agg_write_stream.awaitTermination(1)

:: loading settings :: url = jar:file:/Users/bses/F1_Intern/Kafka_Streaming/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/bses/.ivy2/cache
The jars for the packages stored in: /Users/bses/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-21e28300-5ead-4179-8ea4-c8d31dea606e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 220ms :: artifacts dl 6ms
	:: mo

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



False

In [3]:
# Write the results of query to dataframe
df = spark.sql("SELECT * FROM temp_stream_data")
df.show()

+--------+--------------+------+----------------+---------+
|order_id|account_number|branch|transaction_code|timestamp|
+--------+--------------+------+----------------+---------+
+--------+--------------+------+----------------+---------+



In [ ]:
# Load the values from dataframe df to stream_data table in kafka_con database
from utils.mysql_connection import *
df_table(df.toPandas(),'kafka_con','stream_data')